In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import pyecharts
print(pyecharts.__version__)

import jieba
import jieba.analyse

from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType
from pyecharts import options as opts 
from pyecharts.charts import Map 
from pyecharts.charts import Pie

1.8.1


In [2]:
df = pd.read_csv('茶叶.csv', header=0)

In [3]:
df.head()

,web,image,price,deal,title,shop,location
0,https://click.simba.taobao.com/cc_im?p=%B2%E8%...,//g-search1.alicdn.com/img/bao/uploaded/i4/img...,¥158.00,104人付款,天福茗茶 特级安溪铁观音 2020春茶 乌龙茶叶礼盒装 铁观音茶礼,天福茗茶,福建 厦门
1,//detail.tmall.com/item.htm?id=616174890852&ad...,//g-search2.alicdn.com/img/bao/uploaded/i4/i3/...,¥88.00,440人付款,包邮艺福堂茶叶2020新茶浙香高山云雾毛尖日照正宗250g散装绿茶,天猫超市,上海
2,//detail.tmall.com/item.htm?id=614489362462&ad...,//g-search1.alicdn.com/img/bao/uploaded/i4/i1/...,¥29.90,113人付款,八马茶叶 安溪铁观音清香型兰花香特级品鉴装75克,八马旗舰店,广东 深圳
3,//detail.tmall.com/item.htm?id=35466663421&ns=...,//g-search2.alicdn.com/img/bao/uploaded/i4/i4/...,¥128.00,535人付款,武夷星茶业大红袍礼盒装400g武夷山大红袍武夷岩茶乌龙茶茶叶散装,武夷星旗舰店,福建 南平
4,//detail.tmall.com/item.htm?id=585310589478&ns...,//g-search1.alicdn.com/img/bao/uploaded/i4/i3/...,¥260.00,21人付款,武夷星岩香大红袍茶叶礼盒装500g武夷岩茶乌龙茶武夷山大红袍散装,武夷星旗舰店,福建 南平


### 数据清洗

In [4]:
df['province'] = df.location.apply(lambda x: x.split()[0])
df['city'] = df.location.apply(lambda x: x.split()[0] if len(x) < 4 else x.split()[1])

In [5]:
df['price'] = df.price.str.strip('¥').astype(float)

In [6]:
df['sales'] = df.deal.map(lambda x: int(float(x.strip('万+人付款'))*10000) if '万' in x else int(x.strip('+人付款')))

In [7]:
df.head()

,web,image,price,deal,title,shop,location,province,city,sales
0,https://click.simba.taobao.com/cc_im?p=%B2%E8%...,//g-search1.alicdn.com/img/bao/uploaded/i4/img...,158.0,104人付款,天福茗茶 特级安溪铁观音 2020春茶 乌龙茶叶礼盒装 铁观音茶礼,天福茗茶,福建 厦门,福建,厦门,104
1,//detail.tmall.com/item.htm?id=616174890852&ad...,//g-search2.alicdn.com/img/bao/uploaded/i4/i3/...,88.0,440人付款,包邮艺福堂茶叶2020新茶浙香高山云雾毛尖日照正宗250g散装绿茶,天猫超市,上海,上海,上海,440
2,//detail.tmall.com/item.htm?id=614489362462&ad...,//g-search1.alicdn.com/img/bao/uploaded/i4/i1/...,29.9,113人付款,八马茶叶 安溪铁观音清香型兰花香特级品鉴装75克,八马旗舰店,广东 深圳,广东,深圳,113
3,//detail.tmall.com/item.htm?id=35466663421&ns=...,//g-search2.alicdn.com/img/bao/uploaded/i4/i4/...,128.0,535人付款,武夷星茶业大红袍礼盒装400g武夷山大红袍武夷岩茶乌龙茶茶叶散装,武夷星旗舰店,福建 南平,福建,南平,535
4,//detail.tmall.com/item.htm?id=585310589478&ns...,//g-search1.alicdn.com/img/bao/uploaded/i4/i3/...,260.0,21人付款,武夷星岩香大红袍茶叶礼盒装500g武夷岩茶乌龙茶武夷山大红袍散装,武夷星旗舰店,福建 南平,福建,南平,21


### 数据可视化
#### 提取标题中关键字并制作词云

In [8]:
txt = df['title'].str.cat(sep='。')

# 添加关键词
jieba.add_word('茶叶')

# 读入停用词表
stop_words = []
with open('stop_words.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        stop_words.append(line.strip())

# 添加停用词
stop_words.extend(['10', '100', '250g', '100g', '140g', '130g', '2020', '500g', '500', '750g',\
                   '正宗', '2499g', '特级', '长辈', '特级','礼品','高档', '新茶', '散装', '礼盒装', '礼盒', '罐装',\
                  '送礼','茶叶罐','提购','组合','春茶','高质量','小包装'])  

# 评论字段分词处理
word_num = jieba.analyse.extract_tags(txt,
                                      topK=100,
                                      withWeight=True,
                                      allowPOS=())

# 去停用词
word_num_selected = []

for i in word_num:
    if i[0] not in stop_words:
        word_num_selected.append(i)

key_words = pd.DataFrame(word_num_selected, columns=['words','num'])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\MAYXIA~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.785 seconds.
Prefix dict has been built successfully.


In [9]:
# 词云图
word1 = WordCloud(init_opts=opts.InitOpts(width='1350px', height='750px'))
word1.add("", [*zip(key_words.words, key_words.num)],
          word_size_range=[20, 200],
          shape=SymbolType.DIAMOND)
#          mask_image='茶叶.jpg')
word1.set_global_opts(title_opts=opts.TitleOpts('茶叶商品名称词云图'),
                      toolbox_opts=opts.ToolboxOpts())
word1.render("茶叶商品名称词云图.html")

C:\Users\May Xiao\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'C:\\Users\\May Xiao\\demo\\data_scraping\\Taobao Scraping\\茶叶商品名称词云图.html'

#### 城市总销量地区分布（地图）

In [10]:
# 计算销量
city_num = df.groupby('city')['sales'].sum().sort_values(ascending=False) 

# 绘制地图
map1 = Map(init_opts=opts.InitOpts(width='1350px', height='750px'))
map1.add("", [list(z) for z in zip(city_num.index.tolist(), city_num.values.tolist())],
         maptype="china-cities",
         label_opts=opts.LabelOpts(is_show=False)
        ) 
map1.set_global_opts(title_opts=opts.TitleOpts(title='各城市茶叶销量分布'),
                     visualmap_opts=opts.VisualMapOpts(max_=100000),
                     toolbox_opts=opts.ToolboxOpts()
                    )
map1.render("各城市茶叶销量分布.html")

C:\Users\May Xiao\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'C:\\Users\\May Xiao\\demo\\data_scraping\\Taobao Scraping\\各城市茶叶销量分布.html'

#### 不同价格区间的销量占比

In [11]:
def price_range(x): #按照淘宝推荐划分价格区间
    if x <= 37:
        return '37元以下'
    elif x <= 190:
        return '37-190元'
    elif x <= 368:
        return '190-368元'
    else:
        return '368元以上'

df['price_range'] = df['price'].apply(lambda x: price_range(x)) 
price_cut_num = df.groupby('price_range')['sales'].sum() 

In [12]:
x_data = price_cut_num.index.values.tolist()
y_data = price_cut_num.values.tolist()

(
    Pie(init_opts=opts.InitOpts(width='1350px', height='750px'))
    .add(
        series_name="销量",
        data_pair=[list(z) for z in zip(x_data, y_data)],
        radius=["50%", "70%"],
        label_opts=opts.LabelOpts(formatter='{{b}—占比{d}%}')
    )
    .set_global_opts(legend_opts=opts.LegendOpts(pos_left="left", pos_top='30%',orient="vertical"),
                     toolbox_opts=opts.ToolboxOpts(),
                    title_opts=opts.TitleOpts(title='不同价格区间的茶叶销量占比'))
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
        ),
        # label_opts=opts.LabelOpts(formatter="{b}: {c}")
    )
    .render("不同价格区间的茶叶销量占比.html")
)

C:\Users\May Xiao\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'C:\\Users\\May Xiao\\demo\\data_scraping\\Taobao Scraping\\不同价格区间的茶叶销量占比.html'